# Customers and Products Analysis Using SQL

## The goal of this project is to analyze data from a sales records database for scale model cars and extract information for decision-making.

## The stores.db database contains eight tables:

#### > Customers: customer data
#### > Employees: all employee information
#### > Offices: sales office information
#### > Orders: customers' sales orders
#### > OrderDetails: sales order line for each sales order
#### > Payments: customers' payment records
#### > Products: a list of scale model cars
#### > ProductLines: a list of product line categories

## Project Guide Questions

### Question 1: Which products should we order more of or less of?
### Question 2: How should we tailor marketing and communication strategies to customer behaviors?
### Question 3: How much can we spend on acquiring new customers?

In [5]:
# import necessary libraries

import pandas as pd
import numpy as np
import sqlite3 as sql

In [6]:
# connect to database 

db = sql.connect('stores.db')

In [7]:
cursor = db.cursor()

### Overview of the tables contained in database

In [8]:
cursor.execute("""SELECT 'Customers' AS table_name, 
                         13 AS number_of_attribute,
                         COUNT(*) AS number_of_row
                    FROM Customers
  
                UNION ALL

                SELECT 'Products' AS table_name, 
                       9 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Products

                UNION ALL

                SELECT 'ProductLines' AS table_name, 
                       4 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM ProductLines

                UNION ALL

                SELECT 'Orders' AS table_name, 
                       7 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Orders

                UNION ALL

                SELECT 'OrderDetails' AS table_name, 
                       5 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM OrderDetails

                UNION ALL

                SELECT 'Payments' AS table_name, 
                       4 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Payments

                UNION ALL

                SELECT 'Employees' AS table_name, 
                       8 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Employees

                UNION ALL

                SELECT 'Offices' AS table_name, 
                       9 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Offices;""")
results = cursor.fetchall()
print(results)

[('Customers', 13, 122), ('Products', 9, 110), ('ProductLines', 4, 7), ('Orders', 7, 326), ('OrderDetails', 5, 2996), ('Payments', 4, 273), ('Employees', 8, 23), ('Offices', 9, 7)]


In [9]:
#creating a datadrame to get an organized table
df = pd.read_sql_query("""SELECT 'Customers' AS table_name, 
                         13 AS number_of_attribute,
                         COUNT(*) AS number_of_row
                    FROM Customers
  
                UNION ALL

                SELECT 'Products' AS table_name, 
                       9 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Products

                UNION ALL

                SELECT 'ProductLines' AS table_name, 
                       4 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM ProductLines

                UNION ALL

                SELECT 'Orders' AS table_name, 
                       7 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Orders

                UNION ALL

                SELECT 'OrderDetails' AS table_name, 
                       5 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM OrderDetails

                UNION ALL

                SELECT 'Payments' AS table_name, 
                       4 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Payments

                UNION ALL

                SELECT 'Employees' AS table_name, 
                       8 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Employees

                UNION ALL

                SELECT 'Offices' AS table_name, 
                       9 AS number_of_attribute,
                       COUNT(*) AS number_of_row
                  FROM Offices""", db)
df

,table_name,number_of_attribute,number_of_row
0,Customers,13,122
1,Products,9,110
2,ProductLines,4,7
3,Orders,7,326
4,OrderDetails,5,2996
5,Payments,4,273
6,Employees,8,23
7,Offices,9,7


### Definitions:

##### The low stock represents the quantity of each product sold divided by the quantity of product in stock. We can consider the ten lowest rates. These will be the top ten products that are (almost) out-of-stock.

##### The product performance represents the sum of sales per product.

##### Priority products for restocking are those with high product performance that are on the brink of being out of stock.

In [14]:
# low stock
df = pd.read_sql_query("""SELECT productCode, 
                                 ROUND(SUM(quantityOrdered) * 1.0 / (SELECT quantityInStock
                                                                       FROM products p
                                                                      WHERE od.productCode = p.productCode), 2) AS low_stock
                            FROM orderdetails od
                           GROUP BY productCode
                           ORDER BY low_stock
                           LIMIT 10;""", db)
df

,productCode,low_stock
0,S18_1984,0.09
1,S24_3432,0.09
2,S12_2823,0.10
3,S12_3380,0.10
4,S18_1589,0.10
5,S18_2325,0.10
6,S18_2870,0.10
7,S18_3482,0.10
8,S32_2206,0.10
9,S700_2466,0.10


In [11]:
# product performance
df = pd.read_sql_query("""SELECT productCode, 
                                 SUM(quantityOrdered * priceEach) AS prod_perf
                            FROM orderdetails od
                        GROUP BY productCode 
                        ORDER BY prod_perf DESC
                           LIMIT 10;""", db)
df

,productCode,prod_perf
0,S18_3232,276839.98
1,S12_1108,190755.86
2,S10_1949,190017.96
3,S10_4698,170686.00
4,S12_1099,161531.48
5,S12_3891,152543.02
6,S18_1662,144959.91
7,S18_2238,142530.63
8,S18_1749,140535.60
9,S12_2823,135767.03


In [13]:
# priority Products for restocking
df = pd.read_sql_query("""WITH 
                          low_stock_table AS (
                          SELECT productCode, 
                                 ROUND(SUM(quantityOrdered) * 1.0/(SELECT quantityInStock
                                                                     FROM products p
                                                                    WHERE od.productCode = p.productCode), 2) AS low_stock
                            FROM orderdetails od
                           GROUP BY productCode
                           ORDER BY low_stock
                           LIMIT 10
                           )

                          SELECT productCode, 
                                 SUM(quantityOrdered * priceEach) AS prod_perf
                            FROM orderdetails od
                           WHERE productCode IN (SELECT productCode
                                                   FROM low_stock_table)
                           GROUP BY productCode 
                           ORDER BY prod_perf DESC
                           LIMIT 10;""", db)
df

,productCode,prod_perf
0,S12_2823,135767.03
1,S18_3482,121890.60
2,S18_1984,119050.95
3,S18_2325,109992.01
4,S18_1589,101778.13
5,S18_2870,100770.12
6,S12_3380,98718.76
7,S700_2466,89347.80
8,S24_3432,87404.81
9,S32_2206,33268.76


### Question 2 involves categorizing customers: finding the VIP (very important person) customers and those who are less engaged.

#### VIP customers bring in the most profit for the store.

#### Less-engaged customers bring in less profit.

#### The company could organize some events to drive loyalty for the VIPs and launch a campaign for the less engaged.

In [15]:
# revenue by customer
df = pd.read_sql_query("""SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
                            FROM products p
                            JOIN orderdetails od
                              ON p.productCode = od.productCode
                            JOIN orders o
                              ON o.orderNumber = od.orderNumber
                           GROUP BY o.customerNumber;""", db)
df

,customerNumber,revenue
0,103,10063.80
1,112,31312.72
2,114,70311.07
3,119,60875.30
4,121,41391.52
...,...,...
93,486,33598.57
94,487,17230.12
95,489,10868.04
96,495,25244.69


In [16]:
# top 5 VIPs
df = pd.read_sql_query("""WITH 
                          money_in_by_customer_table AS (
                          SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
                            FROM products p
                            JOIN orderdetails od
                              ON p.productCode = od.productCode
                            JOIN orders o
                              ON o.orderNumber = od.orderNumber
                           GROUP BY o.customerNumber
                        )

                         SELECT contactLastName, contactFirstName, city, country, mc.revenue
                           FROM customers c
                           JOIN money_in_by_customer_table mc
                             ON mc.customerNumber = c.customerNumber
                          ORDER BY mc.revenue DESC
                          LIMIT 5;""", db)
df

,contactLastName,contactFirstName,city,country,revenue
0,Freyre,Diego,Madrid,Spain,326519.66
1,Nelson,Susan,San Rafael,USA,236769.39
2,Young,Jeff,NYC,USA,72370.09
3,Ferguson,Peter,Melbourne,Australia,70311.07
4,Labrune,Janine,Nantes,France,60875.30


In [17]:
# top 5 less engaged
df = pd.read_sql_query("""WITH 
                          money_in_by_customer_table AS (
                          SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
                            FROM products p
                            JOIN orderdetails od
                              ON p.productCode = od.productCode
                            JOIN orders o
                              ON o.orderNumber = od.orderNumber
                           GROUP BY o.customerNumber
                           )

                          SELECT contactLastName, contactFirstName, city, country, mc.revenue
                            FROM customers c
                            JOIN money_in_by_customer_table mc
                              ON mc.customerNumber = c.customerNumber
                           ORDER BY mc.revenue
                           LIMIT 5;""", db)
df

,contactLastName,contactFirstName,city,country,revenue
0,Young,Mary,Glendale,USA,2610.87
1,Taylor,Leslie,Brickhaven,USA,6586.02
2,Ricotti,Franco,Milan,Italy,9532.93
3,Schmitt,Carine,Nantes,France,10063.80
4,Smith,Thomas,London,UK,10868.04


### Question 3 involves checking if it's worth spending money on acquiring new customers

In [19]:
# number of new customers arriving each month
df = pd.read_sql_query("""WITH 
                          payment_with_year_month_table AS (
                          SELECT *, 
                                 CAST(SUBSTR(paymentDate, 1,4) AS INTEGER)*100 + CAST(SUBSTR(paymentDate, 6,7) AS INTEGER) AS year_month
                            FROM payments p
                        ),

                          customers_by_month_table AS (
                          SELECT p1.year_month, COUNT(*) AS number_of_customers, SUM(p1.amount) AS total
                            FROM payment_with_year_month_table p1
                           GROUP BY p1.year_month
                        ),

                           new_customers_by_month_table AS (
                           SELECT p1.year_month, 
                                  COUNT(*) AS number_of_new_customers,
                                  SUM(p1.amount) AS new_customer_total,
                                 (SELECT number_of_customers
                                    FROM customers_by_month_table c
                                    WHERE c.year_month = p1.year_month) AS number_of_customers,
                                 (SELECT total
                                    FROM customers_by_month_table c
                                   WHERE c.year_month = p1.year_month) AS total
                             FROM payment_with_year_month_table p1
                            WHERE p1.customerNumber NOT IN (SELECT customerNumber
                                                              FROM payment_with_year_month_table p2
                                                             WHERE p2.year_month < p1.year_month)
                            GROUP BY p1.year_month
                        )

                         SELECT year_month, 
                                ROUND(number_of_new_customers*100/number_of_customers,1) AS number_of_new_customers_props,
                                ROUND(new_customer_total*100/total,1) AS new_customers_total_props
                         FROM new_customers_by_month_table;""", db)
df

,year_month,number_of_new_customers_props,new_customers_total_props
0,200301,100.0,100.0
1,200302,100.0,100.0
2,200303,100.0,100.0
3,200304,100.0,100.0
4,200305,100.0,100.0
5,200306,100.0,100.0
6,200307,75.0,68.3
7,200308,66.0,54.2
8,200309,80.0,95.9
9,200310,69.0,69.3


### The number of clients has been decreasing since 2003, and in 2004, the company had the lowest values. The year 2005, which is present in the database as well, isn't present in the table above, this means that the store has not had any new customers since September of 2004. This means it makes sense to spend money acquiring new customers.

### To determine how much money we can spend acquiring new customers, we can compute the Customer Lifetime Value (LTV), which represents the average amount of money a customer generates. We can then determine how much we can spend on marketing.

In [18]:
# Customer Lifetime Value (LTV)
df = pd.read_sql_query("""WITH 
                          money_in_by_customer_table AS (
                          SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
                            FROM products p
                            JOIN orderdetails od
                              ON p.productCode = od.productCode
                            JOIN orders o
                              ON o.orderNumber = od.orderNumber
                           GROUP BY o.customerNumber
                           )

                          SELECT AVG(mc.revenue) AS ltv
                            FROM money_in_by_customer_table mc;""", db)
df

,ltv
0,39039.594388


## Summary

### Question 1: Which products should we order more of or less of? 
#### Answer: Classic cars are the priority for restocking. They sell frequently, and they are the highest-performance products.

### Question 2: How should we match marketing and communication strategies to customer behaviors?
#### Answer: Now that we have the most-important and least-committed customers, we can determine how to drive loyalty and attract more customers.

### Question 3: How much can we spend on acquiring new customers?
#### Answer: LTV tells us how much profit an average customer generates during their lifetime with our store. We can use it to predict our future profit. So, if we get ten new customers next month, we'll earn 390,395 dollars, and we can decide based on this prediction how much we can spend on acquiring new customers.

cursor.close()
db.close()